In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torch.optim as optim 
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# Set up for getting reproducibility of result
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# Hyperparameters
n_classes = 10
learning_rate = 0.001
batch_size = 64
n_epochs = 50

In [ ]:
# Transformations
custom_transforms = transforms.Compose([
                                transforms.Resize((32,32)),
                                transforms.ToTensor()
])

In [ ]:
# Load 'MNIST' dataset
train_dataset = datasets.FashionMNIST(root='/content/drive/MyDrive/Datasets/FashionMNIST', train=True, 
                                                    transform=custom_transforms, download=False)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.FashionMNIST(root='/content/drive/MyDrive/Datasets/FashionMNIST', train=False, 
                                                  transform=custom_transforms, download=False)

test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
train_dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: /content/drive/MyDrive/Datasets/FashionMNIST
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )

In [ ]:
test_dataset

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: /content/drive/MyDrive/Datasets/FashionMNIST
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )

In [ ]:
class LeNet5(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.Flatten(),
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
            nn.Softmax(dim=1)
        )
    def forward(self, X):
        prob = self.model(X)
        return prob

In [ ]:
model = LeNet5(n_classes=n_classes).to(device)
model

LeNet5(
  (model): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): Tanh()
    (11): Linear(in_features=84, out_features=10, bias=True)
    (12): Softmax(dim=1)
  )
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(n_epochs):
    for batch_idx, (images, labels) in enumerate(train_loader):
        # Get data to GPU
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        z_scores = model(images)
        loss = criterion(z_scores, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent
        optimizer.step()

        if(batch_idx + 1) % 100 == 0:
            print(f'Epoch {epoch+1}/{n_epochs}, Batch {batch_idx+1}, Loss: {loss.item():.4f}')

In [ ]:
def get_accuracy(loader, model):
    if loader.dataset.train:
        print('Getting accuracy on trainning data')
    else:
        print('Getting accuracy on testing data')

    n_corrects = 0
    n_samples = 0

    # Put model to evalution mode
    model.eval()

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            z_scores = model(images)

            _, y_pred = z_scores.max(1)
            n_corrects += (y_pred == labels).sum()
            n_samples += y_pred.size(0)

    print(f'We got {n_corrects}/{n_samples} correct')
    print(f'Accuracy =  {float(n_corrects) / float(n_samples)*100.0:.2f}')
    
    # Put model to train mode
    model.train()

In [ ]:
get_accuracy(train_loader, model)
get_accuracy(test_loader, model)

Getting accuracy on trainning data
We got 54602/60000 correct
Accuracy =  91.00
Getting accuracy on testing data
We got 8781/10000 correct
Accuracy =  87.81
